In [ ]:
import os
import time
from pathlib import Path

import numpy as np
import tomli
from generate_data import sample_init_state

from models import ModelConfig
from system import SimulationConfig, simulate_system
from unicycle import Unicycle

base_path = Path(os.getcwd()).parent

Params

In [ ]:
config = 'configs/config.toml'
abs_config_path = (base_path / config).resolve()
with open(abs_config_path, 'rb') as f:
    data = tomli.load(f)
    simulation_cfg = SimulationConfig(**data['simulation'])

Evaluation

In [ ]:
model_names = ['Numerical', 'FNO', 'DeepONet']
time_steps = [0.01, 0.005, 0.001]

perf_table = np.zeros((len(model_names), len(time_steps)))
l2_norm_table = np.zeros((len(model_names), len(time_steps)))
model_params = dict()

num_trajectories = 1
for i, model_name in enumerate(model_names):
    for j, time_step in enumerate(time_steps):
        simulation_cfg.dt = time_step
        
        # Construct model config
        model_cfg = None
        if model_name != 'Numerical':
            D1, D2 = simulation_cfg.delays
            time_step_str = f'{time_step}'.split('.')[1]
            model_path = (base_path / f'models/UnicycleConstDistinctDelays_{model_name}_{int(D1 * 100)}_{int(D2 * 100)}_dt_{time_step_str}.pth').resolve()
            model_cfg = ModelConfig(model_name, model_path)

        # Evaluate model performance
        total_time = 0
        total_trajectory_error = 0
        for _ in range(num_trajectories):
            
            # Get new initial condition for each trajectory
            init_state = sample_init_state()
            init_inputs = [np.zeros(NDi, dtype=np.float32) for NDi in simulation_cfg.NDs]

            # Simulation
            ml_unicycle = Unicycle(init_state, init_inputs, simulation_cfg, model_cfg)
            if model_cfg is not None:
                ml_unicycle.use_only_ml_predictor()
                model_params[model_name] = ml_unicycle.get_model_params()

            # Computation time
            start_time = time.time()
            states, _, _, _ = simulate_system(ml_unicycle)
            end_time = time.time()

            total_time += (end_time - start_time)

            # L2 trajectory error
            squared_magnitudes = np.sum(states**2, axis=1)
            total_trajectory_error += np.sqrt(np.trapezoid(squared_magnitudes, dx=time_step))

        # Average
        perf_table[i, j] = total_time / num_trajectories
        l2_norm_table[i, j] = total_trajectory_error / num_trajectories


# Model parameters
print("PARAMETERS")
for name, number in model_params.items():
    print(f"{name:>15}" + f"{number:>12d}")


# Print perf table
print("\nRAW CALCULATION TIMES (ms)")

header = f"{'Step size':>15}" + "".join([f"{ts:>12.4f}" for ts in time_steps])
print(header)
print("-" * len(header))

# Print each row
for i, name in enumerate(model_names):
    print(f"{name:>15}" + "".join([f"{perf:12.4f}" for perf in perf_table[i]]))


# Print L2 trajectory error
print('\nAVERAGE L2 TRAJECTORY ERROR')

header = f"{'Step size':>15}" + "".join([f"{ts:>12.4f}" for ts in time_steps])
print(header)
print("-" * len(header))

for i, name in enumerate(model_names):
    print(f"{name:>15}" + "".join([f"{error:12.4f}" for error in l2_norm_table[i]]))